**Welcome to this Notebook!**  

First, you need to clone this project in Google Colab.

### Steps:  
1. **Open Google Colab**  
   - Go to https://colab.research.google.com 

2. **Open the Notebook**  
   - Click on **File** → **Open Notebook**.  
   - Select the **GitHub** tab and paste the following link into the field:  
     https://github.com/WMallon/Mesa.git

3. **Select the Notebook**  
   - Choose **Tutorial2_glm.ipynb** from the repository.  

4. **Download the Data File**  
   - Download the file to your PC from this link:  
     https://drive.google.com/file/d/17WNyBegtgmuhhNm0N4Z6kbJj5i-8UI0j/view?usp=sharing

5. **Upload the Data File**  
   - Upload it as a ZIP file inside your Colab working folder.  

6. **Unzip the File**  
   - Use the following command to unzip the folder:
   - !unzip sub-01.zip
     
7. **Install the nilearn libraries**  
   - Use the following command to do it:
   - pip install nilearn

In [ ]:
# Import common libraries used during the computation
import os
import numpy as np
import nibabel as nib # nibabel library
import pandas as pd # panda library
import matplotlib as mpl # plotting library
import matplotlib.pyplot as plt # plotting library
# Import Nilearn library
from nilearn.glm import first_level
from nilearn.plotting import plot_design_matrix, plot_contrast_matrix, plot_stat_map
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.glm import threshold_stats_img
from nilearn.image import mean_img

---
### **The Hemodynamic Response Function (HRF) in fMRI**  

The **Hemodynamic Response Function (HRF)** is a mathematical model that describes how neural activity leads to changes in **blood oxygenation** and **blood flow**, which can be measured using functional MRI (**fMRI**). The HRF represents the **time course of the blood-oxygen-level-dependent (BOLD) signal** following a brief burst of neural activity.  

---

### **1. The Process Behind the HRF**  
When a brain region becomes active:  
1. **Neurons fire**, consuming oxygen.  
2. **Local blood flow increases**, delivering more oxygenated blood than is consumed.  
3. The excess oxygenated blood changes the **ratio of oxygenated to deoxygenated hemoglobin**.  
4. This change alters the **BOLD signal**, which is what fMRI detects.  

The HRF characterizes this entire process, typically showing three main phases:  

| Phase | Description |
|-------|------------|
| **Initial Dip** (optional) | A brief **drop in BOLD signal** due to immediate oxygen consumption. |
| **Peak Response** (~4–6s) | A **rise in BOLD signal** as blood flow increases. |
| **Post-Stimulus Undershoot** (~10–30s) | A slight **drop below baseline** before returning to normal. |

--- 

Different fMRI software (like **SPM**, **FSL**, or **AFNI**) may use slightly different HRF models.

---

### **3. Why is the HRF Important in fMRI Analysis?**  
✅ **Models the delayed nature of the BOLD response** (~4–6s lag).  
✅ Helps in **designing experimental paradigms** (e.g., adjusting stimulus timing).  
✅ Used in **statistical analyses** (General Linear Model, GLM) to detect brain activation.  
✅ Essential for **deconvolution** methods in event-related fMRI studies.  

---

### **4. Variability in the HRF**  
The HRF is not identical across all brain regions or individuals. It can vary due to:  
- **Age & Health** (e.g., vascular differences).  
- **Brain Region** (e.g., visual cortex vs. prefrontal cortex).  
- **Stimulus Duration & Type** (e.g., brief vs. sustained stimuli).  

For this reason, some studies use **individualized HRFs** instead of the standard model.

---
OpenAI. ChatGPT (Feb 11 version). 2025. https://chat.openai.com.

In [ ]:
# TO-DO (5 Minutes)
# Search for the HRF on Google and examine its temporal behavior

---
The **Glover** and **SPM HRF (Hemodynamic Response Function)** are two commonly used models for describing the blood-oxygen-level-dependent (**BOLD**) response in functional MRI (**fMRI**). While they serve a similar purpose—modeling how neural activity leads to measurable changes in blood flow—they have key differences in their formulation and characteristics.

---

### **1. Glover HRF**
- Proposed by **Glover (1999)** as an empirical model for the hemodynamic response.
- Defined as a **difference of two gamma functions**:  
  - A **positive** gamma function modeling the main **BOLD response (oxygenated blood flow increase)**.
  - A **negative** gamma function modeling the **undershoot** (post-activation dip).
- More **data-driven** and optimized for empirical fits to fMRI data.

### **2. SPM HRF**
- The **Statistical Parametric Mapping (SPM)** HRF is the default model used in **SPM (a widely used fMRI analysis toolbox)**.
- It is also based on a **double gamma function** but is more standardized.
- The parameters were optimized based on **averaged BOLD responses** from experimental data.

### **Key Differences**
| Feature         | Glover HRF | SPM HRF |
|---------------|-----------|---------|
| **Model Type** | Empirical, fitted to data | Standardized function used in SPM |
| **Flexibility** | More flexible for individual fits | Fixed parameters, good for group studies |
| **Post-Stimulus Undershoot** | Explicitly modeled | Included but less flexible |
| **Use Case** | Custom modeling and optimizing HRF for specific datasets | Standard fMRI analyses, GLM-based methods |

### **Which One to Use?**
- If you are using **SPM or FSL**, the **SPM HRF** is the standard choice.
- If you want a more **customizable HRF** that can better fit individual differences, **Glover HRF** is a better option.

Would you like a visual comparison of their shapes? 😊

In [ ]:
# Show difference between Glover and SPM HRF
time_length = 30

glover_timecourse = first_level.glover_hrf(1, oversampling=50, time_length=time_length)
spm_timecourse = first_level.spm_hrf(1, oversampling=50, time_length=time_length)

sample_times = np.linspace(0, time_length, num=len(glover_timecourse))

plt.plot(sample_times, glover_timecourse, label="Glover")
plt.plot(sample_times, spm_timecourse, label="SPM")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (AU)")
plt.legend()

In [ ]:
# How to compute the Regressors

#Define some functions
def generate_stim(onset, amplitude, duration, hrf_model, maxtime=30):
    # Generate signal with specified duration and onset
    frame_times = np.linspace(0, maxtime, 601)
    exp_condition = np.array((onset, duration, amplitude)).reshape(3, 1)
    stim = np.zeros_like(frame_times)
    stim[(frame_times > onset) * (frame_times <= onset + duration)] = amplitude

    signal, name = first_level.compute_regressor(
        exp_condition, hrf_model, frame_times, con_id="main", oversampling=16
    )

    return frame_times, stim, signal


def plot_regressor(onset, amplitude, duration, hrf_model):
    frame_times, stim, signal = generate_stim(onset, amplitude, duration, hrf_model)
    plt.fill(frame_times, stim, "k", alpha=0.5, label="stimulus")
    plt.plot(frame_times, signal.T[0], label="Regressor")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (AU)")
    plt.legend(loc=1)
    plt.title(hrf_model)
    return None


# Generate an event of 1 second duration starting at time 0
# You could use one of the following HRFs:
# spm, spm + derivative, spm + derivative + dispersion, fir, 
# glover, glover + derivative, glover + derivative + dispersion

hrf_model = "spm"
onset, amplitude, duration = 0.0, 1.0, 1.0
plot_regressor(onset, amplitude, duration, hrf_model)

In [ ]:
# TO-DO (5 Minutes)
# Now change the duration of the stimulus up to 20 s and look for the differences

In Statistical Parametric Mapping (SPM), time and dispersion derivatives are used in fMRI analysis to improve the modeling of the hemodynamic response function (HRF). Their main purpose is to account for variations in the timing and shape of the HRF across different brain regions and subjects.

1. Time Derivative
Captures small variations in the onset of the hemodynamic response.
Useful when there is temporal jitter (slight shifts in timing) between stimulus presentation and the actual BOLD response.
Mathematically, it is the first derivative of the canonical HRF.
Improves model fit without increasing the number of regressors too much.

3. Dispersion Derivative
Captures variations in the duration or width of the HRF.
Useful when the shape of the HRF deviates from the canonical model due to vascular differences or neural adaptation.
Mathematically, it is the second derivative of the canonical HRF.
Helps account for differences in BOLD response duration across brain regions.


When to Use Them?
If you suspect variability in HRF timing, include the time derivative.
If you expect variations in HRF shape, include both time and dispersion derivatives.
They allow for a more flexible and robust GLM analysis in SPM, reducing misalignment effects.

In [ ]:
#TO-DO
# Compare the different HRFs in SPM by overlaying them
# If you need more information, check out this webpage https://nilearn.github.io/dev/modules/glm.html


spm_time = first_level.spm_time_derivative(1, oversampling=50, time_length=time_length)
spm_dispers = first_level.spm_dispersion_derivative(1, oversampling=50, time_length=time_length)
spm_canonical = first_level.spm_hrf(1, oversampling=50, time_length=time_length)

sample_times = np.linspace(0, time_length, num=len(glover_timecourse))
plt.plot(sample_times, spm_timecourse, label="SPM")
plt.plot(sample_times, spm_time, label="Time Derivative")
plt.plot(sample_times, spm_dispers, label="Dispersion Derivative")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (AU)")
plt.legend()

In [ ]:
# Here we plot multiple durations (1,3,5 s...) to see how the resulting regressor varies
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 7))
cmap = mpl.cm.viridis
norm = mpl.colors.Normalize(vmin=0, vmax=40)

for n in [1, 3, 5, 10, 15, 20, 25, 30, 35]:
    frame_times, stim, signal = generate_stim(
        onset, amplitude, n, hrf_model, maxtime=50
    )
    axes.plot(frame_times, signal.T[0], label="Regressor", c=cmap(norm(n)))

axes.set_xlabel("Time (s)")
axes.set_ylabel("Amplitude (AU)")
plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axes)

In [ ]:
# TO-DO (5 Minutes)
# What conclusion can we draw about the HRF duration after examining this plot?

It is time to plot the HRF of a real finger tapping experiment.

A finger-tapping experiment is a commonly used motor task in neuroscience and cognitive research, particularly in functional MRI (fMRI) and EEG studies, to investigate motor control, brain activation, and neuroplasticity.

Overview of the Experiment:
Participants are instructed to tap their fingers in a specific sequence or rhythm (e.g., alternating between index and middle fingers).
The task is often performed under different conditions: self-paced, externally cued (e.g., metronome-paced), or involving different hand movements.
Brain activity is recorded, typically using fMRI to assess activation in motor-related regions (e.g., the primary motor cortex (M1), premotor cortex, supplementary motor area (SMA), and cerebellum).
Purpose:
To study motor system function and connectivity.
To investigate plasticity in patients with neurological disorders (e.g., Parkinson’s disease, stroke recovery).
To assess brain lateralization in motor tasks.


Scientific Reference:
Biswal, B., Yetkin, F. Z., Haughton, V. M., & Hyde, J. S. (1995). Functional connectivity in the motor cortex of resting human brain using echo-planar MRI. Magnetic Resonance in Medicine, 34(4), 537-541. https://doi.org/10.1002/mrm.1910340409

This study is significant because it introduced resting-state functional connectivity, using a finger-tapping task to demonstrate correlations between different motor regions.

You can find a picture of the FT experiment inside the sub-01 folder

Our experiment lasts 400 s

In [ ]:
# TO-DO (20 Minutes)
# Plot the hrf of the finger tapping right, left and baseline separately or superimposed
# Consider the duration of the stimulus



In [ ]:
# Some Imports
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from scipy.stats import pearsonr

In [ ]:
# Now we will demonstrate the FT Block vs Event-Related Design

# Define the random generator
np.random.seed(2)

N = 400 # Total Experiment length in s

# Define the SPM Canonical HRF
dg_hrf = spm_hrf(t_r=2, oversampling=1) # TR = 2

# Definition of the Regressors in the Block Design
# Finger tapping Right
blocked_pred1_onsets = list(range(20, 40)) + list(range(100, 120)) + list(range(180, 200)) + list(range(260, 280)) \
+ list(range(340, 360))   

# Finger tapping Left
blocked_pred2_onsets = list(range(60, 80)) + list(range(140, 160)) + list(range(220, 240)) + list(range(300, 320)) \
+ list(range(380, 400))

# Finger tapping Baseline
blocked_pred3_onsets = list(range(0, 20)) + list(range(40, 60)) + list(range(80, 100)) + list(range(120, 140))\
+ list(range(160, 180)) + list(range(200, 220)) + list(range(240, 260)) + list(range(280, 300)) \
+ list(range(320, 340)) + list(range(360, 380))   #Baseline

# Fill the regressors with ones (according to the experiment timing)
N_stim = len(blocked_pred1_onsets)
blocked_pred1, blocked_pred2, blocked_pred3 = np.zeros(N), np.zeros(N), np.zeros(N)
blocked_pred1[blocked_pred1_onsets] = 1
blocked_pred2[blocked_pred2_onsets] = 1
blocked_pred3[blocked_pred3_onsets] = 1

# Define the Design Matrix X (by convolving the Regressors with the Canonical HRF)
icept = np.ones((N, 1))
X_blocked = np.hstack((
    np.convolve(blocked_pred1, dg_hrf)[:N, np.newaxis],
    np.convolve(blocked_pred2, dg_hrf)[:N, np.newaxis],
    np.convolve(blocked_pred3, dg_hrf)[:N, np.newaxis],
     icept
))

plt.figure(figsize=(20, 10))

plt.subplot(2, 2, 1)
plt.title("Blocked events (red = right, blue = left) without baseline (for clarity)", fontsize=20)
plt.xlim(0, N)
plt.axhline(0, c='tab:blue')
plt.grid()

for onset in blocked_pred1_onsets:
    plt.plot((onset, onset), (0, 1), c='tab:red') # Right

for onset in blocked_pred2_onsets:
    plt.plot((onset, onset), (0, 1), c='tab:blue') # Left
    
''' # We avoid plotting also the baseline         
for onset in blocked_pred3_onsets:
    plt.plot((onset, onset), (0, 1), c='tab:green') # Baseline
'''

plt.subplot(2, 2, 3)
plt.xlim(0, N)
plt.title("Convolved predictors (BLOCKED) with baseline (green)", fontsize=20)
plt.ylim(-1, 2)
plt.plot(X_blocked[:, 0], c='tab:red')
plt.plot(X_blocked[:, 1], c='tab:blue')
plt.plot(X_blocked[:, 2], c='tab:green')
plt.grid()
plt.xlabel("Time (volumes)", fontsize=15)

# Event-Related Design code------------------------------------------------
er_stims = np.arange(N)
er_pred1_onsets = np.random.choice(er_stims, N_stim, replace=False)
er_stims_new = np.array([o for o in er_stims if o not in er_pred1_onsets])
er_pred2_onsets = np.random.choice(er_stims_new, N_stim, replace=False)
er_pred1, er_pred2 = np.zeros(N), np.zeros(N)
er_pred1[er_pred1_onsets] = 1
er_pred2[er_pred2_onsets] = 1

plt.subplot(2, 2, 2)
plt.xlim(0, N)
plt.title("Event onsets (EVENT-RELATED)", fontsize=20)
plt.axhline(0, c='tab:blue')
plt.grid()

for onset in er_pred1_onsets:
    plt.plot((onset, onset), (0, 1), c='tab:blue')

for onset in er_pred2_onsets:
    plt.plot((onset, onset), (0, 1), c='tab:orange')

X_er = np.hstack((
    icept,
    np.convolve(er_pred1, dg_hrf)[:N, np.newaxis],
    np.convolve(er_pred2, dg_hrf)[:N, np.newaxis]
))

plt.subplot(2, 2, 4)

plt.title("Convolved predictors (EVENT-RELATED)", fontsize=20)
plt.ylim(-1, 2)
plt.plot(X_er[:, 1], c='tab:blue')
plt.plot(X_er[:, 2], c='tab:orange')
plt.axhline(0, ls='--', c='k')
plt.xlim(0, N)
plt.grid()
plt.xlabel("Time (volumes)", fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
# It is time to calculate the correlation and to compare them (Block vs Event)
corr_blocked = pearsonr(X_blocked[:, 0], X_blocked[:, 1])
corr_er = pearsonr(X_er[:, 1], X_er[:, 2])
print("Correlation blocked: %.3f. Correlation event-related: %.3f" % (corr_blocked[0], corr_er[0]))

In [ ]:
# TO-DO (10 Minutes)
# Calculate the correlation between the baseline and the left/right regressors


In [ ]:
# TO-DO (5 Minutes)
# Describe the design efficiency?

In [ ]:
# Define the Contrast R > 0
cvec = np.array([1, 0, 0, 0])

In [ ]:
# efficiency = 1.0 / c @ inv(X.T @ X) @ c.T
eff = 1.0 / cvec.dot(np.linalg.inv(X_blocked.T.dot(X_blocked))).dot(cvec.T)
print("Efficiency: %.3f" % (eff))

In [ ]:
# TO-DO (20 Minutes)
# Calculate the Efficiency of the ER Design Matrix

---
In **SPM (Statistical Parametric Mapping)**, the **`rp_` file** contains **realignment parameters** for an fMRI session. These parameters are crucial for **motion correction** in fMRI preprocessing.  

### **1. What is the `rp_` file?**  
- The filename follows this pattern:  
  **`rp_<functional_image>.txt`**  
  - Example: `rp_func_scan1.txt`  
- It is a **text file** generated during **motion correction (realignment)** in SPM.  

---

### **2. What Data Does It Contain?**  
The `rp_` file contains a **6-column matrix**, where each row corresponds to a scan (volume) in the fMRI time series:  

| Column | Parameter | Description |
|---------|------------|----------------|
| 1 | X | Translation (Left/Right) |
| 2 | Y | Translation (Forward/Backward) |
| 3 | Z | Translation (Up/Down) |
| 4 | Pitch | Rotation around **X-axis** |
| 5 | Roll | Rotation around **Y-axis** |
| 6 | Yaw | Rotation around **Z-axis** |

All values are in **millimeters (mm) for translations** and **radians for rotations**.  

---

### **3. Why is the `rp_` File Important?**  
✅ **Motion Correction** – Used to correct for participant head movements.  
✅ **Confound Regressors** – Motion parameters can be added to the **GLM** (General Linear Model) as nuisance regressors to reduce motion-related artifacts.  
✅ **Quality Control** – Helps check if excessive motion (>1–2mm) might affect results.  

---

### **4. How to Use the `rp_` File?**  
- **Include in GLM Analysis**  
  - Motion parameters can be added as **covariates** in the first-level analysis to reduce motion-related confounds.  

---

In [ ]:
# Now, we are going to use the real dataset for the Finger Tapping Experiment
# Inspect the Rigid Body Motion file (rp_**)

# Load the motion file
rigib_body_file = 'sub-01/func/rp_sub-01_task-ft_run-1_bold.txt'
motion_params = np.loadtxt(rigib_body_file)
rotation_params = motion_params[:, :3]
translation_params = motion_params[:, 3:]

# Plot
plt.figure(figsize=(15, 7))
plt.subplot(2, 1, 1)
plt.title('Rotation', fontsize=25)
plt.plot(rotation_params)
plt.xlim(0, motion_params.shape[0])
plt.legend(['x', 'y', 'z'], fontsize=15)
plt.ylabel('Rotation in radians', fontsize=15)
plt.grid()

plt.subplot(2, 1, 2)
plt.title('Translation', fontsize=25)
plt.plot(translation_params)
plt.legend(['x', 'y', 'z'], fontsize=15)
plt.ylabel('Translation in mm', fontsize=15)
plt.xlim(0, motion_params.shape[0])
plt.xlabel('Time (TR)', fontsize=15)
plt.grid()

plt.tight_layout()
plt.show()

---
### **The Design Matrix in SPM?**  

In **SPM (Statistical Parametric Mapping)**, the **design matrix** is a fundamental part of the **General Linear Model (GLM)** used for analyzing fMRI data. It defines how experimental conditions, confounds, and other factors are modeled in the statistical analysis of brain activity.

---

### **1. Purpose of the Design Matrix**  
The design matrix:  
✅ **Models the experimental conditions** (e.g., task vs. rest, stimulus onsets).  
✅ **Accounts for confounds** (e.g., motion, physiological noise).  
✅ **Relates brain activity to predictors** using the **General Linear Model (GLM)**.  
✅ **Helps compute statistical contrasts** to find significant brain activations.  

---

### **2. Structure of the Design Matrix**  
The design matrix is a **2D matrix** where:  
- **Rows** = Individual time points (or scans).  
- **Columns** = Predictors (e.g., task conditions, motion parameters).  

Each column represents a **regressor** that models a specific aspect of the data.  

#### **Example: A Simple Block Design**
| Time Point | Task | Rest | Motion X | Motion Y |
|------------|------|------|---------|---------|
| 1          | 1    | 0    | 0.2     | 0.1     |
| 2          | 1    | 0    | 0.3     | 0.1     |
| 3          | 0    | 1    | -0.1    | 0.2     |
| ...        | ...  | ...  | ...     | ...     |

**Key Components:**
- **Condition Regressors** (e.g., "Task" vs. "Rest")  
- **Nuisance Regressors** (e.g., motion parameters from the `rp_` file)  
- **Drift Terms** (to model slow scanner drifts)  

---

### **3. Visualization of the Design Matrix in SPM**  
SPM provides a **graphical representation** of the design matrix.  
- **Black/white blocks** show experimental conditions.  
- **Gray-scale intensities** represent continuous regressors (e.g., motion parameters).  

---

### **4. How is the Design Matrix Used?**  
SPM fits the GLM:  
\[
Y = X\beta + \epsilon
\]
where:  
- \( Y \) = fMRI data (BOLD signal at each voxel).  
- \( X \) = Design matrix (predictors).  
- \( \beta \) = Regression coefficients (effect sizes).  
- \( \epsilon \) = Residual error.  

---

### **5. Why is the Design Matrix Important?**  
✅ **Ensures valid statistical analysis** by properly modeling the experiment.  
✅ **Allows hypothesis testing** using contrasts (e.g., Task > Rest).  
✅ **Helps control confounds** like motion and scanner drifts.  


---
In **SPM12**, the preprocessing steps for fMRI data are designed to prepare raw data for statistical analysis, ensuring that artifacts are minimized and data are aligned properly for analysis. Below are the key preprocessing steps commonly applied in SPM12:

---

### **1. Realignment (Motion Correction)**
- **Purpose**: Correct for head motion during the scanning session.
- **How it works**: This step involves **aligning** all functional images to a reference image (typically the first or mean image) to account for small movements of the subject's head.
- **Output**: Realigned images and realignment parameters (`rp_` file) containing translations and rotations (X, Y, Z, pitch, roll, yaw).

---

### **2. Slice Timing Correction**
- **Purpose**: Correct for differences in acquisition times between slices in each functional volume.
- **How it works**: If slices are acquired non-sequentially (common in fMRI), slice timing correction adjusts for the time delays by resampling the data, ensuring that all slices are aligned temporally.
- **Output**: Corrected data where the slices are aligned in time.

---

### **3. Coregistration**
- **Purpose**: Align the **structural (anatomical) image** (e.g., T1-weighted) with the functional images.
- **How it works**: The functional images are coregistered (i.e., spatially aligned) with the structural image to ensure that brain regions are in the same space across modalities.
- **Output**: A coregistered functional image set aligned with the structural image.

---

### **4. Segmentation**
- **Purpose**: Separate the structural image into different tissue types (gray matter, white matter, cerebrospinal fluid).
- **How it works**: This step uses tissue probability maps to segment the anatomical image into its components.
- **Output**: Tissue probability maps (gray matter, white matter, CSF) and normalized tissue images.

---

### **5. Normalization (Spatial Normalization)**
- **Purpose**: Map individual subject data to a standard brain template (e.g., MNI space).
- **How it works**: Functional and anatomical images are transformed to fit into a **common space** (e.g., Montreal Neurological Institute (MNI) space). This allows for group analyses by aligning brain structures across individuals.
- **Output**: Normalized images in standard space (e.g., MNI or Talairach space).

---

### **6. Smoothing**
- **Purpose**: Increase the signal-to-noise ratio (SNR) and make the data more Gaussian in distribution.
- **How it works**: A **Gaussian filter** is applied to the images, smoothing the data spatially by averaging nearby voxel values. This step is essential for statistical inference, especially when performing group-level analysis.
- **Output**: Smoothed functional images.

---

### **7. (Optional) Artifact Detection and Detrending**
- **Purpose**: Detect and remove outliers or artifacts, such as motion spikes, or slow drift over time.
- **How it works**: Various methods, such as **motion scrubbing** or **temporal filtering**, may be used to remove these artifacts from the data.
- **Output**: Cleaned and artifact-free data.

---

### **8. (Optional) Temporal Filtering**
- **Purpose**: Remove low- and high-frequency noise from the data.
- **How it works**: A **bandpass filter** can be applied to focus on the frequencies of interest (e.g., 0.01-0.1 Hz for typical resting-state fMRI studies). This helps remove high-frequency noise and low-frequency drifts.
- **Output**: Filtered data focused on the frequency range of interest.

---

### **Summary of Typical Preprocessing Steps in SPM12**:
1. **Realignment** – Correct for head motion.
2. **Slice Timing Correction** – Adjust for slice acquisition timing differences.
3. **Coregistration** – Align functional and structural images.
4. **Segmentation** – Segment the structural image into tissue types.
5. **Normalization** – Transform images to a standard space (e.g., MNI).
6. **Smoothing** – Spatially smooth the functional data.
7. **Artifact Detection/Detrending** (Optional) – Remove outliers or slow drifts.
8. **Temporal Filtering** (Optional) – Remove noise in temporal data.

---

These preprocessing steps are typically followed by statistical modeling using the General Linear Model (GLM) in SPM to identify regions of the brain that are significantly activated during experimental tasks or conditions.

Would you like further clarification on any of these steps? 😊


In [ ]:
# Define the Design Matrix

# Load the pre-processed data (Realigned, Slice-Time corrected, Co-Registerd, Normalized and Smoothed)
# swar..sub-01_task-ft_run-1_bold.nii.gz
# s = smoothed
# w = normalized
# a = STC
# r = realigned

fmri_file = 'sub-01/func/swarsub-01_task-ft_run-1_bold.nii.gz'
f_img = nib.load(fmri_file)
# Get the data
f_img_data = f_img.get_fdata()

# Generate the mean functional image. It will be used to superimpose the functional results.
m_img = mean_img(f_img, copy_header=True)


t_r = 2 #TR = 2
n_scans = 200 #Number of scans = Experiment lenght = 400

frame_times = np.arange(n_scans) * t_r #Time Frames

# Definition of the different trials
conditions = ["B", "R", "B", "L", "B", "R", "B", "L", "B", "R", "B", "L", "B", "R", "B", "L", "B", "R", "B", "L"]
duration = [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
# these are the corresponding onset times
onsets = [0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0, 220.0, 240.0, 260.0, 280.0, 300.0, 320.0, 340.0, 360.0, 380.0]
# Next, we simulate 6 motion parameters jointly observed with fMRI acquisitions
motion = motion_params
# The 6 parameters correspond to three translations and three
# rotations describing rigid body motion
add_reg_names = ["tx", "ty", "tz", "rx", "ry", "rz"]


events = pd.DataFrame({"trial_type": conditions, "onset": onsets, "duration": duration})

# We include three drift regressors. Their aim is to remove signal components unrelated to the expected neural response.

X1 = make_first_level_design_matrix(
    frame_times, events, drift_model='polynomial', drift_order=3,
    add_regs=motion, add_reg_names=add_reg_names, hrf_model=hrf_model)



In [ ]:
plot_design_matrix(X1)

In [ ]:
# Define the GLM
fmri_glm = FirstLevelModel(
    smoothing_fwhm=5,
    minimize_memory=True,
)

In [ ]:
# Fit the GLM
fmri_glm = fmri_glm.fit(fmri_file, design_matrices=X1)

In [ ]:
# Define the Contrast
n_regressors = X1.shape[1]
activation = np.zeros(n_regressors)
activation[1] = 1
activation[2] = -1

In [ ]:
plot_contrast_matrix(contrast_def=activation, design_matrix=X1)

In [ ]:
# Compute the contrast
eff_map = fmri_glm.compute_contrast(activation, output_type="effect_size")

In [ ]:
# Compute the t statistics contrast
t_map = fmri_glm.compute_contrast(activation, stat_type = "t", output_type="stat")
#z_map = fmri_glm.compute_contrast(activation, output_type="z_score")

In [ ]:
# Show the activations
cut_coords = [42, 40, 21]
plotting_config = {
    "bg_img": m_img,
    "display_mode": "z", #Axial
    "cut_coords": cut_coords,
    "black_bg": True,
}
plot_stat_map(
    t_map,
    threshold=3,
    title="right > left (|t|>5)",
    figure=plt.figure(figsize=(10, 4)),
    **plotting_config,
)

In [ ]:
# Show the activations using the FDR control
clean_map, threshold = threshold_stats_img(
    t_map, alpha=0.05, height_control="fdr", two_sided=False
)
plot_stat_map(
    clean_map,
    threshold=threshold,
    title=(
        f"right > rest (p<0.05 FDR-corrected; threshold: {threshold:.3f})"
    ),
    figure=plt.figure(figsize=(10, 4)),
    **plotting_config,
)